## Enviando docs para banco de dados no AWS

In [ ]:
#Importando Modulos e conectando com a base via pymysql
import pymysql
import pandas as pd
import csv

host = 'database-1.cyvfcyenu8c2.us-east-1.rds.amazonaws.com'
port = 3306
user = #USER
password = #PASSWORD
db_name='database-1'

cnxn = pymysql.connect(host = host,
                       user = user,
                       database=db_name,
                       password = password)

In [9]:
#Tratando o arquivo Vendas csv. Subsitui a string na coluna revenue $ por "" 
#Isso nos deixa tratar o valor como int
#Tambem manipulamos a data e deixamops no formato dd/mm/yyyy
#Depois salvamos odf manipulado de volta para um novo csv de vendas

df = pd.read_csv('dir_local' + '\vendas.csv',sep=',')
df.head()

df["revenue"]= df["revenue"].str.replace("$","")
df["revenue"]= df["revenue"].str.replace(",","")
df["date"]= df["date"].str.replace("12:00:00 AM","")
df.to_csv('dir_local' + '\Vendas_new.csv', index=False)  
df.dtypes

productid     int64
date         object
zip           int64
units         int64
revenue      object
dtype: object

In [ ]:
#Porximas celulas irao ler o arquivo e transformaos em dataframes. Cada arquivo representa uma tabela da base de dados

In [ ]:
Fabricante=pd.read_csv('dir_local' + '\fabricante.csv',sep=',')

In [ ]:
Geo=pd.read_csv('dir_local' + 'geo.csv',sep=';')

In [3]:
Produto=pd.read_csv('dir_local' + '\produto.csv',sep=',')

In [ ]:
Vendas=pd.read_csv('dir_local' + '\Vendas_new.csv',sep=',')

In [ ]:
#Definimos o cursos da conexao com a base de dados com parametros cnxn aacima
mycursor = cnxn.cursor()

In [ ]:
#Demos um replace nos valores vazios de Revenue por zero. 
Vendas["revenue"].fillna(0, inplace = True) 

As proximas celulas criam as queries utilizadas para subir as tabelas no banco de dados 

In [ ]:
sql_Create_Fabricante='''CREATE TABLE Fabricante(
ID INT NOT NULL,
manufacturerid INT NOT NULL,
manufacturer VARCHAR(255) NULL,
PRIMARY KEY (ID)
);'''

In [ ]:
sql_Create_Geo='''CREATE TABLE Geo(
zip INT NULL,
city VARCHAR(255) NULL,
state VARCHAR(255) NULL,
region VARCHAR(255) NULL,
district VARCHAR(255) NULL
);'''

In [ ]:
sql_Create_Produto='''CREATE TABLE Produto(
category varchar(255) NULL,
segment varchar(255) NULL,
product varchar(255) NULL,
productid int NOT NULL,
iscompetehide varchar(255) NULL,
manufacturerid int NOT NULL,
PRIMARY KEY (productid)
);'''

In [ ]:
sql_Create_Vendas='''CREATE TABLE Vendas(
productid int  NULL,
date varchar(255) NULL,
zip varchar(255) NULL,
units int  NULL,
revenue int NULL
);'''

In [ ]:
# A linha abaixo cria uma query que drop as tabelas da base de dados se elas ja existirem
sql_drop="DROP TABLE IF Exists Fabricante,Geo,Produto,Vendas;"

In [ ]:
# O cursor executa a query drop e depois da um commit na base
#Isso é necessario pois a conexao nao esta em autocommit
mycursor.execute(sql_drop)
mycursor.execute('commit')

In [ ]:
# O cursos executa as queries de criaçao das tabelas na base
mycursor.execute(sql_Create_Fabricante)
mycursor.execute(sql_Create_Produto)
mycursor.execute(sql_Create_Geo)
mycursor.execute(sql_Create_Vendas)
mycursor.execute('commit')

In [ ]:
sql=''

#Inserting DataFrame records one by one.
for i,row in Fabricante.iterrows():
        
    sql += f'({row[0]}, {row[1]},"{row[2]}"),\n'


sql="INSERT INTO Fabricante (ID,manufacturerid,manufacturer)\nVALUES\n" + sql[:-2]+";"

print(sql)

mycursor.execute(sql)

mycursor.execute('commit')

In [ ]:
sql=''

#Inserting DataFrame records one by one.
for i,row in Geo.iterrows():
        
    sql += f'({row[0]}, "{row[1]}","{row[2]}","{row[3]}","{row[4]}"),\n'

sql="INSERT INTO Geo (zip,city,state,region,district)\nVALUES\n" + sql[:-2]+";"

print(sql)

mycursor.execute(sql)

mycursor.execute('commit')

In [ ]:
sql=''

#Inserting DataFrame records one by one.
for i,row in Produto.iterrows():
        
    sql += f'("{row[0]}", "{row[1]}","{row[2]}",{row[3]},"{row[4]}",{row[5]}),\n'


sql="INSERT INTO Produto (category,segment,product,productid,iscompetehide,manufacturerid)\nVALUES\n" + sql[:-2]+";"

print(sql)

mycursor.execute(sql)

mycursor.execute('commit')

In [ ]:
# O for loop abaixo segue a mesma logica das queries acima, porem como a tabela é muito grande precisamos 
#executa-la em pedaços. Para isso utilizamos um contador que zera as linhas da query a cada 10000 linhas
#Em outras palavras inserimos queries de 10k de linhas por "iteraçao" do contador count. 

count=1

sql=''
#Inserting DataFrame records one by one.
for i,row in Vendas.iterrows():
        
    sql += f'({row[0]}, "{row[1]}","{row[2]}",{row[3]},{row[4]}),\n'
    
    if i==10000*count:
    
        sql="INSERT INTO Vendas (productid,date,zip,units,revenue)\nVALUES\n" + sql[:-2]+";"

        mycursor.execute(sql)

        mycursor.execute('commit')

        print(count)

        sql=''

        count=count+1